<a href="https://colab.research.google.com/github/Selinalkan/GitPractice/blob/main/Collocations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
#imports
import nltk
from nltk.collocations import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
import csv
import spacy
import re
import string

from nltk.corpus import webtext
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import stopwords
from nltk.collocations import *

assert nltk.download('averaged_perceptron_tagger')
assert nltk.download('stopwords')

stop = stopwords.words('english')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
fourgram_measures = nltk.collocations.QuadgramAssocMeasures()

bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()
bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(unlist_comments)
#trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(unlist_comments)

In [53]:
nlp = spacy.load("en_core_web_sm")

# Interviewer with Context

In [54]:
#Importing ER_c_Transcript Dataset and dropping the Unnamed column
df = pd.read_csv("ER_c_transcript.csv").drop("Unnamed: 0", axis=1)

In [55]:
#Sorting the data by "speaker_trustworthy" values
df_sorted1 = df.sort_values("speaker_trustworthy", ascending=False)
#Sorting the data by "speaker_trusted by conversational partner" values
df_sorted2 = df.sort_values("speaker_trusts conversational partner", ascending=False)

In [56]:
#extract only ER_text1
ER_text1 = df_sorted1['ER_text'][:10]
ER_text1 = ER_text1.astype('str')

#extract only ER_text2
ER_text2 = df_sorted2['ER_text'][:10]
ER_text2 = ER_text2.astype('str')

In [ ]:
ER_text1

**"speaker_trustworthy"**

In [58]:
#function to remove non-ascii characters
def _removeNonAscii(s): return "".join(i for i in s if ord(i)<128)
#remove non-ascii characters
ER_text1 = ER_text1.map(lambda x: _removeNonAscii(x))

In [60]:
def clean_text(text):
    #remove punctuations
    regex = re.compile('[' + re.escape(string.punctuation) + '\\r\\t\\n]')
    nopunct = regex.sub(" ", str(text))
    #use spacy to lemmatize comments
    doc = nlp(nopunct, disable=['parser','ner'])
    lemma = [token.lemma_ for token in doc]
    return lemma

In [61]:
#apply function to clean and lemmatize comments
lemmatized = ER_text1.map(clean_text)

#make sure to lowercase everything
# lemmatized = lemmatized.map(lambda x: [word.lower() for word in x])

#turn all comments' tokens into one single list
unlist_lemmatized = [item for items in lemmatized for item in items]

In [62]:
#function to filter for ADJ/NN bigrams
def rightTypes(ngram):
    if '-pron-' in ngram or '' in ngram or ' 'in ngram or 't' in ngram:
        return False
    for word in ngram:
        if word in stop:
            return False
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    tags = nltk.pos_tag(ngram)
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

In [63]:
#filter bigrams
filtered_bi = bigramFreqTable[bigramFreqTable.bigram.map(lambda x: rightTypes(x))]

In [64]:
filtered_bi[:10]

,bigram,freq
152,"(Climate, Agreement)",2
151,"(Paris, Climate)",2
159,"(U, S)",2
189,"(good, weapon)",1
185,"(interest, rate)",1
155,"(President, Trump)",1
156,"(Trump, pull)",1
217,"(carbon, dioxide)",1
218,"(dioxide, increase)",1
43,"(attack, use)",1


In [30]:
# PMI
#filter for only those with more than 20 occurences
bigramFinder.apply_freq_filter(30)
bigramPMITable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.pmi)), columns=['bigram','PMI']).sort_values(by='PMI', ascending=False)